#**MUSIC GENRE CLASSIFICATION**

In [1]:
#importing required packages

import os
import numpy as np
import pandas as pd
import scipy
import librosa


##importing dataset GTZAN

In [3]:
#specifying the dataset path for further processing

WAV_DIR = '/content/drive/MyDrive/gtzan/Data/genres_original'
wav_files = os.listdir(WAV_DIR)
print(wav_files, len(wav_files))

['jazz', 'disco', 'reggae', 'country', 'rock', 'hiphop', 'blues', 'classical', 'metal', 'pop'] 10


##Feature Extraction

In [4]:

# creating predefined col name , which process through pandas dataframe, for feature extraction purpose


col_names = ['file_name', 'genre name','zcr_mean', 'zcr_std',
             'spectral_centroid_mean', 'spectral_centroid_std'] +\
                     ['spectral_rolloff_mean', 'spectral_rolloff_std'] + \
            ['mfccs_' + str(i+1) + '_mean' for i in range(20)] + \
            ['mfccs_' + str(i+1) + '_std' for i in range(20)] 
dfppt= pd.DataFrame(columns=col_names)
print(dfppt)



Empty DataFrame
Columns: [file_name, genre name, zcr_mean, zcr_std, spectral_centroid_mean, spectral_centroid_std, spectral_rolloff_mean, spectral_rolloff_std, mfccs_1_mean, mfccs_2_mean, mfccs_3_mean, mfccs_4_mean, mfccs_5_mean, mfccs_6_mean, mfccs_7_mean, mfccs_8_mean, mfccs_9_mean, mfccs_10_mean, mfccs_11_mean, mfccs_12_mean, mfccs_13_mean, mfccs_14_mean, mfccs_15_mean, mfccs_16_mean, mfccs_17_mean, mfccs_18_mean, mfccs_19_mean, mfccs_20_mean, mfccs_1_std, mfccs_2_std, mfccs_3_std, mfccs_4_std, mfccs_5_std, mfccs_6_std, mfccs_7_std, mfccs_8_std, mfccs_9_std, mfccs_10_std, mfccs_11_std, mfccs_12_std, mfccs_13_std, mfccs_14_std, mfccs_15_std, mfccs_16_std, mfccs_17_std, mfccs_18_std, mfccs_19_std, mfccs_20_std]
Index: []

[0 rows x 48 columns]


In [18]:

 #doing feature extraction for only ppt mentioned features

#working m feature extraction and saving it in csvfile




#Humans can detect sounds in a frequency range from about 20 Hz to 20 kHz.
#The sampling rate you use is directly related to the range of frequencies you can sample.
# With a sampling rate of 44,100 Hz, the highest frequency you can sample is 22,050 Hz. But if 20,000 Hz is the 
#upper limit of human hearing, why would you ever need to sample a frequency higher than that? 
#And why do we have digital systems able to work at sampling rates as high as 192,000 Hz?
#thats why sr is taken as 22050
import sys
import os
from fnmatch import fnmatch
#import IPython.display as ipd

root = '/content/drive/MyDrive/gtzan/Data/genres_original/'
pattern = "*.wav"
cnt=0
#print(root)
#print(type(root))
ii=0
#to iterate through every file in subfolder
for path, subdirs, files in os.walk(root):
    for name in files:
        #if(cnt==10):
          #break
        if fnmatch(name, pattern):
            xx=os.path.join(path, name)
            #print(xx)
            
            try:
        # Read wav-file
              y, sr = librosa.load(xx, sr = 22050)

              feature_list = [name]
              temp=name.split('.')
              feature_list.append(temp[0])
              #print(temp)
              #print(type(temp[0]))
              #print(type(feature_list[0]))

              
              zcr = librosa.feature.zero_crossing_rate(y + 0.0001, frame_length=2048, hop_length=512)[0]
              feature_list.append(np.mean(zcr))
              feature_list.append(np.std(zcr))

              
              spectral_centroids = librosa.feature.spectral_centroid(y+0.01, sr=sr)[0]
              feature_list.append(np.mean(spectral_centroids))
              feature_list.append(np.std(spectral_centroids))

              
              spectral_rolloff = librosa.feature.spectral_rolloff(y+0.01, sr=sr, roll_percent = 0.85)[0]
              feature_list.append(np.mean(spectral_rolloff))
              feature_list.append(np.std(spectral_rolloff))

              mfccs = librosa.feature.mfcc(y, sr=sr, n_mfcc=20)
              feature_list.extend(np.mean(mfccs, axis=1))
              feature_list.extend(np.std(mfccs, axis=1))

              
              feature_list[2:] = np.round(feature_list[2:], decimals=3)
              cnt+=1 
              #print(feature_list)
              #if(cnt==10):
                #break

            except:
                e=sys.exc_info()
                print(e)
            dfppt = dfppt.append(pd.DataFrame(feature_list, index=col_names).transpose(), ignore_index=True)
print(dfppt)
# Save file
dfppt.to_csv('dfppt_features.csv', index=False)



/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


(<class 'audioread.exceptions.NoBackendError'>, NoBackendError(), <traceback object at 0x7f83dc57b4b0>)
           file_name genre name zcr_mean zcr_std spectral_centroid_mean  \
0     jazz.00002.wav       jazz    0.058    0.03               1027.444   
1     jazz.00000.wav       jazz    0.078   0.049                 1434.3   
2     jazz.00005.wav       jazz    0.058    0.03                1228.79   
3     jazz.00004.wav       jazz    0.048   0.023               1002.328   
4     jazz.00003.wav       jazz    0.045   0.014                768.097   
...              ...        ...      ...     ...                    ...   
2995   pop.00097.wav        pop    0.137   0.142               3482.045   
2996   pop.00091.wav        pop    0.167   0.099               3676.932   
2997   pop.00095.wav        pop    0.106   0.066               3032.248   
2998   pop.00096.wav        pop    0.105   0.064               3105.976   
2999   pop.00090.wav        pop    0.148   0.088               3578.279

In [13]:

dfppt = pd.read_csv(r'/content/drive/MyDrive/dfppt_featuress.csv')
print (dfppt)

          file_name genre name  zcr_mean  zcr_std  spectral_centroid_mean  \
0    jazz.00002.wav       jazz     0.058    0.030                1027.444   
1    jazz.00000.wav       jazz     0.078    0.049                1434.300   
2    jazz.00005.wav       jazz     0.058    0.030                1228.790   
3    jazz.00004.wav       jazz     0.048    0.023                1002.328   
4    jazz.00003.wav       jazz     0.045    0.014                 768.097   
..              ...        ...       ...      ...                     ...   
995   pop.00097.wav        pop     0.137    0.142                3482.045   
996   pop.00091.wav        pop     0.167    0.099                3676.932   
997   pop.00095.wav        pop     0.106    0.066                3032.248   
998   pop.00096.wav        pop     0.105    0.064                3105.976   
999   pop.00090.wav        pop     0.148    0.088                3578.279   

     spectral_centroid_std  spectral_rolloff_mean  spectral_rolloff_std  \


##splitting the dataset

In [14]:
%matplotlib notebook

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
#first parameter is FORM TO number of rows,second parameter is FROM TO nubmer of columns
X = dfppt.iloc[:,2:98]
print(X)
y = dfppt['genre name']

#default splitting ratio is 75:25
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3)
print(X_train)
print(y_train)

     zcr_mean  zcr_std  spectral_centroid_mean  spectral_centroid_std  \
0       0.058    0.030                1027.444                404.982   
1       0.078    0.049                1434.300                755.763   
2       0.058    0.030                1228.790                536.889   
3       0.048    0.023                1002.328                343.993   
4       0.045    0.014                 768.097                302.496   
..        ...      ...                     ...                    ...   
995     0.137    0.142                3482.045               1502.242   
996     0.167    0.099                3676.932                875.567   
997     0.106    0.066                3032.248                984.001   
998     0.105    0.064                3105.976               1136.390   
999     0.148    0.088                3578.279                967.319   

     spectral_rolloff_mean  spectral_rolloff_std  mfccs_1_mean  mfccs_2_mean  \
0                 1857.210               70

##Normalization

In [15]:
#feature scaling operation performing on xtrain and xtest data , inorder to normalize 
#the data to some range of values .....

from sklearn.preprocessing import StandardScaler
# why fit_transofmr and transform : https://towardsdatascience.com/what-and-why-behind-fit-transform-vs-transform-in-scikit-learn-78f915cf96fe
st_x= StandardScaler()
x_train= st_x.fit_transform(X_train)

x_test= st_x.transform(X_test)

print(x_train)
print(x_test)

[[ 0.99501309  1.1130992   1.13483077 ...  1.41766286  0.68747583
  -0.18635832]
 [ 0.21987955  1.57986084  1.07083575 ... -0.29724938  0.64628448
   0.55990599]
 [-1.86329184 -1.30557473 -1.23252778 ... -1.29765809 -0.35247862
  -0.45545734]
 ...
 [-0.02234968  1.36769646  0.6050435  ...  0.32437388  0.25420386
   0.50765824]
 [ 0.00187324 -0.24475283 -0.18550245 ... -0.64126959 -1.01509996
  -0.88129961]
 [-1.18504999 -1.00854459 -0.57013056 ... -0.32319359  0.10622012
  -0.49337164]]
[[-0.09501845 -0.58421584  0.0092707  ...  0.15781205  0.08130189
  -0.59046923]
 [-1.06393538 -1.13584322 -1.93360274 ...  2.7055334   2.61075589
   3.97173022]
 [-1.23349584 -0.41448433 -0.04179207 ... -0.24847426 -0.23348139
  -0.62977063]
 ...
 [-0.31302476 -1.26314185 -0.99217506 ...  0.41932968 -0.12058657
  -0.38101586]
 [ 0.70433801  1.49499508  0.57027568 ... -0.15040515  0.52525311
  -0.53822148]
 [ 0.07454201 -0.58421584  0.3559954  ... -0.80575588 -0.77456282
   0.62741193]]


##Training using SVM and checking accuracy

In [16]:
#ref: https://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html

import numpy as np
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
clf = OneVsRestClassifier(SVC()).fit(x_train, y_train)
print("accuracy is "+ str(clf.score(x_test,y_test)))

accuracy is 0.684
